In [1]:
import os
import numpy as np
import pandas as pd


In [2]:
from alphanet import AlphaNetV3, load_model
from alphanet.data import TrainValData, TimeSeriesData
from alphanet.metrics import UpDownAccuracy

import pandas as pd

2022-12-17 15:36:48.191212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-17 15:36:49.120839: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-17 15:36:49.120913: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-17 15:36:49.120936: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [3]:
data_folder = "/work/bd/summer2022/insample/datacache"
dailydata = os.path.join(data_folder, 'dailydata') # close, high, low, open, return, tvrvalue, tvrvolume.parquet
labeldata = os.path.join(data_folder, 'labeldata') # 
univdata = os.path.join(data_folder, 'univdata')
xmindata = os.path.join(data_folder, 'xmindata')

In [4]:
df_close = pd.read_parquet(os.path.join(dailydata, 'close.parquet'))
df_high = pd.read_parquet(os.path.join(dailydata, 'high.parquet'))
df_low = pd.read_parquet(os.path.join(dailydata, 'low.parquet'))
df_open = pd.read_parquet(os.path.join(dailydata, 'open.parquet'))
df_tvrvalue = pd.read_parquet(os.path.join(dailydata, 'tvrvalue.parquet'))
df_tvrvolume = pd.read_parquet(os.path.join(dailydata, 'tvrvolume.parquet'))

df_return = pd.read_parquet(os.path.join(dailydata, 'return.parquet'))

In [5]:
df_y0 = pd.read_parquet(os.path.join(labeldata, 'Y_0.parquet'))

In [55]:
stocks = df_open.columns.values
stocks

array(['000001', '000002', '000003', ..., '688800', '688819', '688981'],
      dtype=object)

In [16]:
df_low

,000001,000002,000003,000004,000005,000006,000007,000008,000009,000010,...,688786,688787,688788,688789,688793,688798,688799,688800,688819,688981
date,,,,,,,,,,,,,,,,,,,,,
20180102,13.320000,31.450001,NaN,22.000000,4.15,9.85,16.66,8.70,7.23,5.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180103,13.200000,32.230000,NaN,22.270000,4.22,9.85,16.66,8.68,7.30,5.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180104,13.130000,32.099998,NaN,23.120001,4.23,9.85,16.66,8.58,7.51,5.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180105,13.150000,32.799999,NaN,22.850000,4.26,9.85,16.66,8.50,7.51,5.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180108,12.860000,35.110001,NaN,22.730000,4.27,9.85,16.66,8.40,7.46,5.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,17.160000,19.580000,NaN,18.590000,2.24,4.37,5.66,2.84,13.81,3.71,...,52.000000,83.620003,127.360001,261.630005,91.099998,209.600006,37.650002,136.179993,41.880001,52.529999
20211228,17.090000,19.389999,NaN,18.860001,2.25,4.35,5.65,2.72,13.91,3.76,...,52.500000,85.150002,128.720001,266.679993,98.500000,212.550003,39.259998,137.460007,42.060001,52.700001
20211229,16.700001,19.360001,NaN,19.200001,2.27,4.35,5.67,2.60,14.05,3.72,...,51.660000,83.680000,130.809998,260.000000,99.500000,218.509995,39.599998,137.000000,42.209999,52.619999


In [6]:
import numpy as np

features = [df_open, df_high, df_low, df_tvrvalue, df_tvrvolume, df_close]
stock_data_list = []

stocks = df_open.columns.values
for stock in stocks:
    one_stock_features = []
    for feature in features:
        one_stock_features.append(feature[stock].values[:-2].reshape(-1, 1))
    stock_np_features = np.concatenate(one_stock_features, axis=1)
    dates = feature.index.values[:-2]
    labels = df_y0[stock].values[1:]
    print(stock_np_features.shape, dates.shape, labels.shape)
    stock_data_list.append(TimeSeriesData(dates=dates, data=stock_np_features, labels=labels))
print(len(stocks))

(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (971,) (971,)
(971, 6) (9

In [27]:
stock_data_list[0].dates.shape

(972,)

In [30]:
*(stock_data_list[0].data).shape

SyntaxError: can't use starred expression here (1675315249.py, line 1)

In [28]:
len(np.unique([date for stock in stock_data_list for date in stock.dates]))

972

In [7]:
train_val_data = TrainValData(time_series_list=stock_data_list, train_length=800, validate_length=150, history_length=10)

In [8]:
train, val, dates_info = train_val_data.get(20180102, order='by_date')

data_all shape: (378300, 6, 10)
label_all shape: (378300,)
dates_series_all shape: (378300, 2)
data_all shape: (1997424, 6, 10)
label_all shape: (1997424,)
dates_series_all shape: (1997424, 2)


In [10]:
len(train)

1475283

In [12]:
for data in train:
    if np.isnan(data):
        print(data)

/tmp/ipykernel_171777/2203936312.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if np.isnan(data):


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [38]:
print(len(train))

1475283


In [40]:
train.shape

AttributeError: 'TensorSliceDataset' object has no attribute 'shape'

In [24]:
print(len(val))

327990


<tf.Tensor: shape=(), dtype=float32, numpy=-0.007900001>

In [9]:
type(train)

tensorflow.python.data.ops.from_tensor_slices_op.TensorSliceDataset

In [11]:
next(iter(train.batch(200)))

(<tf.Tensor: shape=(200, 30, 6), dtype=float32, numpy=
 array([[[1.33500004e+01, 1.39300003e+01, 1.33199997e+01, 2.85654374e+09,
          2.08159248e+08, 1.36999998e+01],
         [1.37299995e+01, 1.38599997e+01, 1.31999998e+01, 4.00622080e+09,
          2.96249856e+08, 1.33299999e+01],
         [1.33199997e+01, 1.33699999e+01, 1.31300001e+01, 2.45454362e+09,
          1.85450944e+08, 1.32500000e+01],
         ...,
         [1.28299999e+01, 1.29200001e+01, 1.25299997e+01, 2.71146496e+09,
          2.13781504e+08, 1.25400000e+01],
         [1.20799999e+01, 1.20799999e+01, 1.13800001e+01, 3.30855347e+09,
          2.82494976e+08, 1.16899996e+01],
         [1.17799997e+01, 1.18400002e+01, 1.15600004e+01, 1.43964429e+09,
          1.22878248e+08, 1.17200003e+01]],
 
        [[3.14500008e+01, 3.29900017e+01, 3.14500008e+01, 2.21850266e+09,
          6.83433520e+07, 3.25600014e+01],
         [3.25000000e+01, 3.37799988e+01, 3.22299995e+01, 2.13024973e+09,
          6.46870200e+07, 3.2330001

In [53]:
import numpy as np
a = np.concatenate([df_open['000001'].values.reshape(-1, 1), df_close['000001'].values.reshape(-1, 1)], axis=1)
a.shape

(973, 2)

In [39]:
# get an AlphaNetV3 instance
import tensorflow as tf
model = AlphaNetV3(l2=0.001, dropout=0.0)

# you may use UpDownAccuracy() here to evaluate performance
model.compile(metrics=[tf.keras.metrics.RootMeanSquaredError(),
                       UpDownAccuracy()])

# train
model.fit(train.batch(500).cache(),
          validation_data=val.batch(500).cache(),
          epochs=100)

Epoch 1/100
2951/2951 [==============================] - 21s 6ms/step - loss: 0.0061 - root_mean_squared_error: 0.0776 - up_down_accuracy: 0.5014 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0379 - val_up_down_accuracy: 0.5040
Epoch 2/100
2951/2951 [==============================] - 16s 5ms/step - loss: 0.0011 - root_mean_squared_error: 0.0323 - up_down_accuracy: 0.5023 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0318 - val_up_down_accuracy: 0.5073
Epoch 3/100
2951/2951 [==============================] - 15s 5ms/step - loss: 8.9276e-04 - root_mean_squared_error: 0.0299 - up_down_accuracy: 0.5085 - val_loss: 9.8560e-04 - val_root_mean_squared_error: 0.0314 - val_up_down_accuracy: 0.5105
Epoch 4/100
2951/2951 [==============================] - 16s 5ms/step - loss: 8.8465e-04 - root_mean_squared_error: 0.0297 - up_down_accuracy: 0.5109 - val_loss: 9.8042e-04 - val_root_mean_squared_error: 0.0313 - val_up_down_accuracy: 0.5137
Epoch 5/100
2951/2951 [=========================

In [41]:
# save model by save method
# model.save("test_model.pth")

# or just save weights
model.save_weights("weights_10.pth")

In [10]:
new_model = AlphaNetV3(l2=0.001, dropout=0.0)
new_model.load_weights("weights_.pth")

In [42]:
preds = model.predict(val.batch(500).cache())

663/663 [==============================] - 3s 3ms/step


In [43]:
print(preds.shape)

(331227, 1)


In [44]:
labels = []
for sample in val.batch(500):
    # print(sample[1])
    labels.append(sample[1])
    # break
labels = np.concatenate(labels)

In [45]:
labels.shape

(331227,)

In [46]:
np.save('preds_10.npy', preds.reshape(-1))

In [47]:
np.save('labels_10.npy', labels)

In [ ]:
import matplotlib.pyplot as plt


In [28]:
next(iter(val))[1]
print(len(val.batch(500).cache()))

656
